- Data Ingestion => docs
- Text Splitter => splitted_docs
- Embedding =>
- Retrieval Chain

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['LANGSMITH_API_KEY'] = os.getenv("LANGSMITH_API_KEY")
os.environ['LANGSMITH_TRACING'] = "true"
os.environ['LANGSMITH_PROJECT'] = "Components_Of_Langchain"

### Data Ingestion
- From the website we need to scrap the data

In [2]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://huggingface.co/docs/transformers/index")
docs = loader.load()
docs

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://huggingface.co/docs/transformers/index', 'title': '🤗 Transformers', 'description': 'We’re on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n🤗 Transformers\n\n\n\n\n\n\n\n\nHugging Face\n\n\n\n\n\n\n\n\t\t\t\t\tModels\n\n\t\t\t\t\tDatasets\n\n\t\t\t\t\tSpaces\n\n\t\t\t\t\tPosts\n\n\t\t\t\t\tDocs\n\n\t\t\t\t\tEnterprise\n\nPricing\n\t\t\t\n\n\n\n\n\n\nLog In\n\t\t\t\t\nSign Up\n\t\t\t\t\t\n\n\n\nTransformers documentation\n\t\t\t\n🤗 Transformers\n\n\n\nTransformers\n\n🏡 View all docsAWS Trainium & InferentiaAccelerateAmazon SageMakerArgillaAutoTrainBitsandbytesChat UICompetitionsDataset viewerDatasetsDiffusersDistilabelEvaluateGoogle CloudGoogle TPUsGradioHubHub Python LibraryHugging Face Generative AI Services (HUGS)Huggingface.jsInference API (serverless)Inference Endpoints

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splitted_docs = splitter.split_documents(docs)
splitted_docs

[Document(metadata={'source': 'https://huggingface.co/docs/transformers/index', 'title': '🤗 Transformers', 'description': 'We’re on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.'}, page_content='🤗 Transformers\n\n\n\n\n\n\n\n\nHugging Face\n\n\n\n\n\n\n\n\t\t\t\t\tModels\n\n\t\t\t\t\tDatasets\n\n\t\t\t\t\tSpaces\n\n\t\t\t\t\tPosts\n\n\t\t\t\t\tDocs\n\n\t\t\t\t\tEnterprise\n\nPricing\n\t\t\t\n\n\n\n\n\n\nLog In\n\t\t\t\t\nSign Up\n\t\t\t\t\t\n\n\n\nTransformers documentation\n\t\t\t\n🤗 Transformers\n\n\n\nTransformers\n\n🏡 View all docsAWS Trainium & InferentiaAccelerateAmazon SageMakerArgillaAutoTrainBitsandbytesChat UICompetitionsDataset viewerDatasetsDiffusersDistilabelEvaluateGoogle CloudGoogle TPUsGradioHubHub Python LibraryHugging Face Generative AI Services (HUGS)Huggingface.jsInference API (serverless)Inference Endpoints (dedicated)LeaderboardsLightevalOptimumPEFTSafetensorsSentence Transforme

In [4]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [5]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(splitted_docs,embedding)
db

### Query From vectorDB

In [6]:
query = "Which common tasks are supportted by Transformer models in different modalities?"
result = db.similarity_search(query)
result[0].page_content

'🤗 Transformers State-of-the-art Machine Learning for PyTorch, TensorFlow, and JAX. 🤗 Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. These models support common tasks in different modalities, such as: 📝 Natural Language Processing: text classification, named entity recognition, question answering, language modeling, code generation, summarization, translation, multiple choice, and text generation.\n🖼️ Computer Vision: image classification, object detection, and segmentation.\n🗣️ Audio: automatic speech recognition and audio classification.'

#### Load LLM Model

In [7]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x00000195A49221A0> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000195A4887940> root_client=<openai.OpenAI object at 0x00000195A4922AA0> root_async_client=<openai.AsyncOpenAI object at 0x00000195A49208B0> model_name='gpt-4o' model_kwargs={} openai_api_key=SecretStr('**********')


### Retrieval Chain

In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
"""
Answer the following question based only on the provided context:
<context>
{context}
"""
)

document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000195A49221A0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000195A4887940>, root_client=<openai.OpenAI object at 0x00000195A4922AA0>, root_async_client=<openai.AsyncOpenAI object at 0x00000195A49208B0>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'},

In [12]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"Which common tasks are supportted by Transformer models in different modalities?",
    "context": [Document(page_content="Which common tasks are supportted by Transformer models in different modalities?")]
})

'Transformer models are versatile and support a range of tasks across different modalities. In text or language processing, they support tasks such as translation, text generation, sentiment analysis, and summarization. For vision tasks, they are used in object detection, image classification, and image segmentation. In speech processing, they facilitate tasks like speech recognition and text-to-speech conversion. These models excel in handling and generating data across these modalities due to their ability to capture long-range dependencies and contextual information.'

#### So far the input we are passing directly to LLM model but at first we want it to go through our vectordb. By this way we will first search related information from vector db and that information along with input will be passed to LLM 

In [13]:
## Retriver :  input =>   retriever => vectordb
from langchain.chains import create_retrieval_chain
retriever = db.as_retriever()
retrieval_chain = create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000019537EB7BE0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n'), additional_kwargs={})])
            | ChatOpenAI(clien

In [16]:
## Get the response from LLM
response = retrieval_chain.invoke({"input":"Does transformer supports PyTorch, TensorFlow, and JAX"})
response

{'input': 'Does transformer supports PyTorch, TensorFlow, and JAX',
 'context': [Document(id='1889cc00-4565-4e84-8665-f61550c3b3a5', metadata={'source': 'https://huggingface.co/docs/transformers/index', 'title': '🤗 Transformers', 'description': 'We’re on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.'}, page_content='🤗 Transformers State-of-the-art Machine Learning for PyTorch, TensorFlow, and JAX. 🤗 Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. These models support common tasks in different modalities, such as: 📝 Natural Language Processing: text classification, named entity recognition, question answering, language modeling, code generation, summarization, translation, multiple choice, and text gen

In [18]:
response["answer"]

'Which models support all three frameworks: PyTorch, TensorFlow, and Flax?\n\n- ALBERT\n- BART\n- BARThez\n- BARTpho\n- BERT\n- BertJapanese\n- BERTweet\n- Blenderbot\n- BlenderbotSmall\n- CamemBERT\n- CPM'

In [19]:
response["context"]

[Document(id='1889cc00-4565-4e84-8665-f61550c3b3a5', metadata={'source': 'https://huggingface.co/docs/transformers/index', 'title': '🤗 Transformers', 'description': 'We’re on a journey to advance and democratize artificial intelligence through open source and open science.', 'language': 'No language found.'}, page_content='🤗 Transformers State-of-the-art Machine Learning for PyTorch, TensorFlow, and JAX. 🤗 Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. Using pretrained models can reduce your compute costs, carbon footprint, and save you the time and resources required to train a model from scratch. These models support common tasks in different modalities, such as: 📝 Natural Language Processing: text classification, named entity recognition, question answering, language modeling, code generation, summarization, translation, multiple choice, and text generation.\n🖼️ Computer Vision: image classification, object detection, and segmen